In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
# Create a dash application
app = dash.Dash(__name__)
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                               dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',  # Default to 'ALL' to display all sites initially
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                              html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
                                # TASK 3: Add a slider to select payload range
                                html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=1000,
        marks={i: f'{i}' for i in range(int(min_payload), int(max_payload), 1000)},
        value=[min_payload, max_payload]
    ),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])
# TASK 2:
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        site_data = spacex_df
    else:
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
    success_count = site_data['class'].sum()
    failure_count = len(site_data) - success_count
    fig = px.pie(names=['Success', 'Failure'], values=[success_count, failure_count],
                 title=f"Launch Success vs Failure for {selected_site}" if selected_site != 'ALL' else "Launch Success vs Failure for All Sites")
    return fig
# TASK 4:
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    min_payload, max_payload = payload_range
    filtered_data = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)]
    if selected_site != 'ALL':
        filtered_data = filtered_data[filtered_data['Launch Site'] == selected_site]
    fig = px.scatter(filtered_data, x='Payload Mass (kg)', y='class', color='class', title=f"Payload vs Success for {selected_site}" if selected_site != 'ALL' else "Payload vs Success for All Sites")
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port= 8050)